In [1]:
with open("Day24.txt") as file:
    swires, sgates = file.read().split("\n\n")    

In [2]:
from dataclasses import dataclass

@dataclass
class Gate:
    in1: str
    in2: str
    out: str
    op: str

    def __post_init__(self):
        # Commutative boolean law
        self.in1, self.in2 = sorted((self.in1, self.in2))
    
    def run(self, wires):
        if (out := wires.get(self.out)) is not None:
            return out
        in1, in2 = wires.get(self.in1), wires.get(self.in2)
        if in1 is None or in2 is None:
            return
        match self.op:
            case "AND":
                out = in1 & in2
            case "OR":
                out = in1 | in2
            case "XOR":
                out = in1 ^ in2
        wires[self.out] = out
        return out

In [3]:
import re

wires = {}
for line in swires.splitlines():
    name, value = line.split(": ")
    wires[name] = int(value)

gates = {}
regex = re.compile(r"(?P<in1>\w+) (?P<op>\w+) (?P<in2>\w+) -> (?P<out>\w+)")
for line in sgates.splitlines():
    if match := regex.match(line):
        gates[match.group("out")] = Gate(**match.groupdict())

In [4]:
from queue import deque

def run(wires, gates):
    wires = wires.copy()
    queue = deque(gates.values())
    while queue:
        gate = queue.popleft()
        if gate.run(wires) is None:
            queue.append(gate)
    result = 0
    for wire, value in sorted(wires.items(), reverse=True):
        if not wire.startswith("z"):
            continue
        result = (result << 1) | value
    return result

In [5]:
%%time
run(wires, gates)

CPU times: user 389 μs, sys: 38 μs, total: 427 μs
Wall time: 443 μs


57344080719736

In [6]:
def solve(gates):
    """
    Full-adder circuit:
        Z[n] = ( X[n] XOR Y[n] ) XOR C[n-1]
        C[n] = ( X[n] AND Y[n] ) OR ( ( X[n] XOR Y[n] ) AND C[n-1] )
    """
    defects = set()
    for i in range(1, 45):
        x, y, z = f"x{i:02}", f"y{i:02}", f"z{i:02}"
        zgate = gates[z]
        # Z gates should be XOR only
        if zgate.op != "XOR":
            defects.add(z)
            continue
        lft, rgt = gates[zgate.in1], gates[zgate.in2]
        # Rearrange Z input's gates to have XOR on left and OR on right
        if lft.op == "OR" or rgt.op == "XOR":
            lft, rgt = rgt, lft
        # Left Z input gate should be XOR: ( X[n] XOR Y[n] )
        if lft.op != "XOR":
            defects.add(lft.out)
        else:
            # XOR gates for Z inputs would only have X/Y inputs
            # X/Y input gates must follow Z increment
            if lft.in1 != x:
                defects.add(lft.in1)
            if lft.in2 != y:
                defects.add(lft.in2)
        # Exception: C[1] is just X[1] AND Y[1]
        if rgt.op == "AND" and i == 1:
            continue
        # Right Z input gate should be OR
        elif rgt.op != "OR":
            defects.add(rgt.out)
        else:
            # If OR gate, ensure both inputs come from AND gates
            g1, g2 = gates[rgt.in1], gates[rgt.in2]
            if g1.op != "AND":
                defects.add(rgt.in1)
            if g2.op != "AND":
                defects.add(rgt.in2)
    return ",".join(sorted(defects))

In [7]:
%%time
solve(gates)

CPU times: user 42 μs, sys: 4 μs, total: 46 μs
Wall time: 47.4 μs


'cgq,fnr,kqk,nbc,svm,z15,z23,z39'